In [ ]:
import sys
import json
import pandas as pd
import numpy as np
from tqdm import tqdm  # Used to measure progress of long a running script.

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns when displaying DataFrames.

In [ ]:
drop_list = ['tpk-niputuspvm',
             'ohjaaja_ytunnus_kj_tutkinto',
             'opiskeluoikeus_oid']

rename_dict = {'tyopaikan_normalisoitu_nimi': 'tyopaikka_normalisoitu',
               'tyopaikan_nimi': 'tyopaikka',
               'oppilaitos': 'oppilaitos_oid',
               'koulutustoimija': 'koulutustoimija_oid',
               'alkupvm':'vastaamisajan_alkupvm',
               'viimeinen_vastauspvm': 'vastaamisajan_loppupvm',
               'oppilaitos': 'oppilaitos_oid',
               'uusi-kesto-without-oa': 'tyopaikkajakson_kesto',
               'hankkimistapa_tyyppi': 'sopimustyyppi',
               'tyopaikan_ytunnus': 'tyonantaja',
               'tutkinto': 'tutkintotunnus',
               'tutkinnonosa_nimi': 'paikallinen_tutkinnon_osa',
               'jakso_loppupvm': 'tyopaikkajakson_loppupvm',
               'jakso_alkupvm': 'tyopaikkajakson_alkupvm',
               'tutkinnonosa_koodi': 'tutkinnon_osa',
               'existing-arvo-tunnus': 'tunnus'}

types = {
    "osa_aikaisuus": 'string',
    "tyopaikka": 'string',
    "rahoitusryhma": 'string',
    "tyopaikka_normalisoitu": 'string',
    "vastaamisajan_alkupvm": 'string',
    "tyonantaja": 'string',
    "oppisopimuksen_perusta": 'string',
    "tyopaikkajakson_kesto": 'string',
    "osaamisala": 'string',
    "koulutustoimija_oid": 'string',
    "paikallinen_tutkinnon_osa": 'string',
    "tyopaikkajakson_loppupvm": 'string',
    "toimipiste_oid": 'string',
    "oppilaitos_oid": 'string',
    "tyopaikkajakson_alkupvm": 'string',
    "tutkintotunnus": 'string',
    "sopimustyyppi": 'string',
    "tutkintonimike": 'string',
    "request_id": 'string',
    "vastaamisajan_loppupvm": 'string',
    "tutkinnon_osa": 'string'
}

In [ ]:
with open('jaksotunnus-taulun-data.json') as file:
    data = json.load(file)
    vanha_df = pd.DataFrame(data)
    vanha_df = vanha_df.set_index('hankkimistapa_id')
    vanha_df = vanha_df.drop(columns=drop_list)
    vanha_df = vanha_df.fillna('')
    vanha_df = vanha_df.rename(columns=rename_dict)

with open('rahoituslaskenta-taulun-data.json') as file:
    data = json.load(file)
    uusi_df = pd.DataFrame(data)
    uusi_df = uusi_df.set_index('hankkimistapa_id')
    uusi_df = uusi_df.drop(columns=drop_list)
    uusi_df = uusi_df.drop(columns=['vanha-kesto', 'uusi-kesto-with-oa', 'save-timestamp', 'tallennuspvm'])
    uusi_df = uusi_df.fillna('')
    uusi_df = uusi_df.rename(columns=rename_dict)

In [ ]:
# Suodatetaan jaksot joissa kesto yli 1500 päivää.
uusi_df = uusi_df[uusi_df.tyopaikkajakson_kesto <= 1500]
# Suodatetaan valitut jaksot
# uusi_df = uusi_df[~uusi_df.index.isin([...])]

In [ ]:
def get_tutkinnonosa(x):
    splitted = x.split('_')
    if len(splitted) == 2:
        return splitted[1]
    return x

def to_comma_separated_list(x):
    # Yksinkertaiseksi listaksi, jossa elementit eroteltu pilkulla.
    return ','.join(map(lambda y: y[1:-1], x[1:-1].split()))
    # Sama kuin yllä, paitsi ympäröity lisäksi hakasulkeilla.
    # return ','.join(x.replace('(', '[').replace(')', ']').split())
    
def get_tutkinnonosa(x):
    splitted = x.split('_')
    if len(splitted) == 2:
        return splitted[1]
    return x

In [ ]:
# tepRahoitusTablen jaksot joille ei ole vielä löydetty Arvo-tunnusta hankkimistapa-id:n perusteella.
uusi_without_tunnus = uusi_df[uusi_df.tunnus == '']
# Kopio yllä olevasta DataFramesta. Tätä täytetään alla olevassa silmukassa.
tunnukset_found = uusi_without_tunnus
# Jaksot jaksotunnus-taulusta, joista on poistettu ne jaksot jotka on jo mätsätty hankkimistapa-id:n perusteella.
vanha_unmatched = vanha_df[~vanha_df.tunnus.isin(uusi_df.tunnus[uusi_df.tunnus != ''])]
# Suodatetaan pois uuden rahoituskauden jaksot.
vanha_unmatched = vanha_unmatched[vanha_unmatched.tyopaikkajakson_loppupvm < "2022-07-01"]
# Uniikit HOKS ID:t listaan.
unique_hoks_ids = np.unique(uusi_df['hoks_id'])

for hoks_id in tqdm(unique_hoks_ids):
    
    # Haetaan hoks_id:tä vastaavat rivit (jaksot) jaksotunnus-taulun datasta
    # (`vanha_df`) ja tepRahoitusLaskenta-taulun datasta (`uusi_df`).
    rows_uusi = uusi_without_tunnus[uusi_without_tunnus.hoks_id == hoks_id]
    rows_vanha = vanha_unmatched[vanha_unmatched.hoks_id == hoks_id]
    
    fields = ['tyonantaja',
              'tutkinnon_osa',
              'tyopaikkajakson_loppupvm',
              'tyopaikkajakson_alkupvm',
              'ohjaaja_email',
              'ohjaaja_nimi',
              'ohjaaja_puhelinnumero',
              'sopimustyyppi',
              'tutkinnonosa_tyyppi',
              'toimipiste_oid']
        
    for index,row in rows_uusi.iterrows():
        
        match_found = False                   
        rows_trimmed = rows_vanha
        rows_before_trimming = rows_trimmed
        
        for field in fields:
                
            rows_before_trimming = rows_trimmed
            rows_trimmed = rows_trimmed[rows_trimmed[field] == row[field]]
            rows_left = rows_trimmed.shape[0]

            # * Jos jäljelle jääneitä rivejä on vain yksi, voidaan olla varmoja että
            #   rivin `row` jakso vastaa jäljelle jääneen rivin jaksoa.
            if rows_left == 1:
                tunnukset_found.at[index, 'tunnus'] = rows_trimmed.tunnus.values[0]
                match_found = True
                break
                
            # Jos trimmattiin liikaa, ts. kaikki loput rivit, palautetaan aikaisemmat
            # rivit takaisin jotta voidaan testata muita vertailukriteerejä seuraavilla
            # iteraatiokerroilla.
            if rows_left == 0:
                rows_trimmed = rows_before_trimming
 
        if not match_found:
            tunnukset_found.at[index, 'tunnus'] = ','.join(rows_trimmed.tunnus.values)

In [ ]:
paatellyt = tunnukset_found[(tunnukset_found.tunnus != '')]

# Poistetaan jaksot joissa yhdelle vastaajatunnukselle on useampi jaksokandidaatti
varmat_ja_useampi_tunnus = paatellyt[~paatellyt.tunnus.isin(paatellyt[paatellyt.duplicated(subset=['tunnus'])].tunnus)]
assert not varmat_ja_useampi_tunnus.duplicated(subset=['tunnus']).any()

varmat = varmat_ja_useampi_tunnus[~varmat_ja_useampi_tunnus.tunnus.str.contains(',')]
useampi_tunnus = varmat_ja_useampi_tunnus[varmat_ja_useampi_tunnus.tunnus.str.contains(',')]

assert varmat_ja_useampi_tunnus.shape[0] == varmat.shape[0] + useampi_tunnus.shape[0]

duplikaatit = paatellyt[paatellyt.tunnus.isin(paatellyt[paatellyt.duplicated(subset=['tunnus'])].tunnus)]
duplikaatit_yksi_tunnus = duplikaatit[~duplikaatit.tunnus.str.contains(',')]
duplikaatit_useampi_tunnus = duplikaatit[duplikaatit.tunnus.str.contains(',')]

assert paatellyt.shape[0] == duplikaatit.shape[0] + varmat.shape[0] + useampi_tunnus.shape[0]
assert duplikaatit[duplikaatit.tunnus == ''].empty

varmat = pd.concat([varmat, uusi_df[uusi_df.tunnus != '']])
assert not varmat.duplicated(subset=['tunnus']).any() # Ei pitäisi sisältää yhtään duplikaattia
assert not useampi_tunnus.duplicated(subset=['tunnus']).any() # Ei pitäisi sisältää yhtään duplikaattia

assert varmat[varmat.tunnus == ''].empty # Ei pitäisi pitää sisällään tunnuksettomia jaksoja.

loput = uusi_df[~uusi_df.index.isin(pd.concat([varmat, duplikaatit, useampi_tunnus]).index)]
assert loput[loput.tunnus != ''].empty # Pitäisi pitää sisällään pelkästää tyhjiä merkkijonoja (ei tunnusta).

# Varmistetaan, että juuri määritettyjen DataFramejen yhteenlaskettu rivimäärä vastaa `uusi_df` DataFramen rivimäärää.
assert uusi_df.shape[0] == varmat.shape[0] + duplikaatit_yksi_tunnus.shape[0] + duplikaatit_useampi_tunnus.shape[0] + useampi_tunnus.shape[0] + loput.shape[0]

# Tarkistetaan, ettei eri DataFramejen tunnukset leikkaa.
assert not varmat.tunnus.isin(duplikaatit_yksi_tunnus.tunnus).any()
assert not varmat.tunnus.isin(duplikaatit_useampi_tunnus.tunnus).any()
assert not varmat.tunnus.isin(useampi_tunnus.tunnus).any()
assert not duplikaatit_yksi_tunnus.tunnus.isin(useampi_tunnus.tunnus).any()
assert not duplikaatit_yksi_tunnus.tunnus.isin(duplikaatit_useampi_tunnus.tunnus).any()
assert not duplikaatit_useampi_tunnus.tunnus.isin(useampi_tunnus.tunnus).any()

In [ ]:
def to_csv(df, name):
    df = df.astype(types)
    df['tutkinnon_osa'] = df['tutkinnon_osa'].apply(get_tutkinnonosa)
    df['tutkintonimike'] = df['tutkintonimike'].apply(to_comma_separated_list)
    df['osaamisala'] = df['osaamisala'].apply(to_comma_separated_list)
    df = df.drop(columns=['niputuspvm', 'oppija_oid','hoks_id', 'rahoituskausi', 'ohjaaja_nimi','ohjaaja_email', 'ohjaaja_puhelinnumero', 'tutkinnonosa_id', 'tutkinnonosa_tyyppi'])
    df = df.set_index('tunnus')
    df.to_csv(name + '.csv', sep='|')

In [ ]:
to_csv(varmat, 'jaksot-varmoilla-tunnuksilla')
to_csv(duplikaatit_yksi_tunnus, 'jaksot-useampi-jakso-yhta-tunnusta-kohden')
to_csv(duplikaatit_useampi_tunnus, 'jaksot-useampi-jakso-samaa-tunnusjoukkoa-kohden')
to_csv(useampi_tunnus, 'jaksot-useampi-tunnus-jaksoa-kohden')
to_csv(loput, 'jaksot-ei-olemassa-tunnusta')